# Requirements + setup

In [1]:
# ! pip install sqlalchemy

In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.ops import unary_union
import folium
from shapely.geometry import mapping
import json
from pyspark.sql import SparkSession
import unidecode
from addressing.utils import libpostal
from fuzzywuzzy import fuzz
import nltk
from nltk.corpus import stopwords
from addressing.automatic_matching import automatic_matching
from addressing.automatic_matching.rooftop.rooftop import haversine_distance
import re
import sys, os
import sqlalchemy
from datetime import date


data_path = "/mnt/c/Users/tandon/OneDrive - TomTom/Desktop/tomtom/Workspace/01_Rooftop_accuracy/BFP_Analysis_USA/data/data"
state = "Texas"
county = 'tarrant'
delta = True
print("Delta :",delta)
Updated_geometries_pickle = os.path.join(data_path,state,county,"Apt_realignment_MSFT/FinalUpdated_APT_Texas_tarrant.pkl")
if os.path.isfile(Updated_geometries_pickle):
    print(Updated_geometries_pickle)

/home/tandon/anaconda3/envs/pygeo/lib/python3.8/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/home/tandon/anaconda3/envs/pygeo/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Delta : True
/mnt/c/Users/tandon/OneDrive - TomTom/Desktop/tomtom/Workspace/01_Rooftop_accuracy/BFP_Analysis_USA/data/data/Texas/tarrant/Apt_realignment_MSFT/FinalUpdated_APT_Texas_tarrant.pkl


In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/tandon/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
DB = {
'host' : "10.137.173.84",
'port' : '5432',
'database' :  "STAN",
'user' : "strategicadmin",
'password' :  "TBmG4Yj3DdwOI+Aq"
}

class ReadAndWrite2PostgresDB:

    def __init__(self, engine):
        self.engine = engine

    def read_from_db(self, query, retry_num=3):

        for _ in range(retry_num):
            df = None
            try:
                df = pd.read_sql(query, self.engine)
                return df

            except Exception as e:
                print(e)

        return df

    def write_to_db(self, df, schema, table_name, retry_num=3):

        for _ in range(retry_num):
            try:
                
                df.to_sql(
                    table_name,
                    con=self.engine,
                    if_exists='append',
                    schema=schema,
                    index = False)
                print("Table stored!")
                return 1

            except Exception as e:
                print(e)

        return 0

In [5]:
engine = sqlalchemy.create_engine(f'postgresql+psycopg2://{DB["user"]}:{DB["password"]}@{DB["host"]}:{DB["port"]}/{DB["database"]}',
                                        echo = False)

In [6]:
raw2p = ReadAndWrite2PostgresDB(engine)

counties = 'cook', 'bexar', 'dallas', 'orange', 'maricopa', 'marion', 'tarrant', 'clark'

In [7]:
date = date.today()
print(date)
print(county)

2022-10-06
tarrant


Stopwords

In [8]:
countries_stopwords = {
  'br': stopwords.words('portuguese') + ['rua', 'avenida'], 
  'ca': stopwords.words('french') + stopwords.words('english') +  ['road', 'street', 'st.', 'st', 'rue', 'chemin', 'avenue'],
  'es': stopwords.words('spanish') + ['calle', 'avenida', 'callejón', 'paseo'],
  'fr': stopwords.words('french') + ['rue', 'chemin', 'avenue'],
  'gb': stopwords.words('english') + ['street', 'road', 'avenue', 'st.', 'st', 'drive'],
  'it': stopwords.words('italian') + ['via', 'viale', 'strada'],
  'mx': stopwords.words('spanish') + ['calle', 'avenida', 'callejón', 'paseo'],
  'us': stopwords.words('english') + ['street', 'road', 'avenue', 'st.', 'st', 'drive'],
  'be': stopwords.words('french') + ['rue', 'chemin', 'avenue'],
  'za': stopwords.words('english') + ['street', 'road', 'avenue', 'st.', 'st', 'drive']
}

countries_stopwords = {k:'|'.join(['\\b' + word + '\\b' for word in v]) for k, v in countries_stopwords.items()}

## Reading the data

### Sample addresses

In [9]:
sample_query = f"""SELECT * FROM "STAN_169".sample where county = '{county}'"""
sample_df = raw2p.read_from_db(query = sample_query)
sample_geom = gpd.GeoSeries.from_wkt(sample_df.geometry)
sample_gdf = gpd.GeoDataFrame(sample_df.drop('geometry', axis = 1),
                                geometry = sample_geom,
                                crs = 'EPSG:4326')

In [10]:
print(sample_gdf.shape)
sample_gdf.head()

(4727, 29)


,country,searched_query_unidecode_sample,libpostal_query,libpostal_response,libpostal_house,libpostal_category,libpostal_near,libpostal_house_number,libpostal_road,libpostal_unit,...,libpostal_state_district,libpostal_state,libpostal_country_region,libpostal_country,libpostal_world_region,lat_sample,lon_sample,county,sample_id,geometry
0,USA,"1450 8th Ave, Fort Worth, TX 76104, USA","{""query"": ""1450 8th Ave, Fort Worth, TX 76104,...","{'country': 'usa', 'city': 'fort worth', 'road...",,,,1450,8th ave,,...,,tx,,usa,,32.728714,-97.344899,tarrant,1ext3ggtarrant,"POLYGON ((-97.33592 32.72871, -97.33596 32.727..."
1,USA,"15225 FAA Blvd, Fort Worth, TX 76155, USA","{""query"": ""15225 FAA Blvd, Fort Worth, TX 7615...","{'country': 'usa', 'city': 'fort worth', 'road...",,,,15225,faa blvd,,...,,tx,,usa,,32.829891,-97.036186,tarrant,2ext3ggtarrant,"POLYGON ((-97.02720 32.82989, -97.02725 32.829..."
2,USA,"3109 Tanglewood Trail, Fort Worth, TX 76109, USA","{""query"": ""3109 Tanglewood Trail, Fort Worth, ...","{'country': 'usa', 'city': 'fort worth', 'road...",,,,3109,tanglewood trail,,...,,tx,,usa,,32.705244,-97.381751,tarrant,3ext3ggtarrant,"POLYGON ((-97.37277 32.70524, -97.37281 32.704..."
3,USA,"5125 Rondo Dr, Fort Worth, TX 76106, USA","{""query"": ""5125 Rondo Dr, Fort Worth, TX 76106...","{'country': 'usa', 'city': 'fort worth', 'road...",,,,5125,rondo dr,,...,,tx,,usa,,32.835560,-97.318310,tarrant,4ext3ggtarrant,"POLYGON ((-97.30933 32.83556, -97.30937 32.834..."
4,USA,"5401 Alliance Gateway Fwy, Fort Worth, TX 7617...","{""query"": ""5401 Alliance Gateway Fwy, Fort Wor...","{'country': 'usa', 'city': 'fort worth', 'road...",,,,5401,alliance gateway fwy,,...,,tx,,usa,,32.986580,-97.246989,tarrant,5ext3ggtarrant,"POLYGON ((-97.23801 32.98658, -97.23805 32.985..."


### Source

In [11]:
source_query = f"""SELECT * FROM "STAN_169".source_v0 where county = '{county}'"""
source_df = raw2p.read_from_db(query = source_query)
source_geom = gpd.GeoSeries.from_wkt(source_df.geometry)
source_gdf = gpd.GeoDataFrame(source_df.drop('geometry', axis = 1),
                                geometry = source_geom,
                                crs = 'EPSG:4326')

In [12]:
source_gdf.head()

,feat_id,postal_code,hsn,state,city,street_name,country_code,prefix,suffix,pre_dir,post_dir,x,y,county,geometry
0,00005554-3100-2800-0000-0000000f4375,76020,1004,TX,Azle,Scotland Ave,USA,None,Ave,None,None,-97.538003,32.873872,tarrant,POINT (-97.53800 32.87387)
1,00005554-3100-2800-0000-00000012efc4,76137,7304,TX,Fort Worth,Big Bend Ct,USA,None,Ct,None,None,-97.268139,32.872685,tarrant,POINT (-97.26814 32.87268)
2,ffd96005-2511-49dc-87b5-25fc04895044,76036,901,TX,Crowley,Rutherford Dr,USA,None,Dr,None,None,-97.336473,32.588961,tarrant,POINT (-97.33647 32.58896)
3,00005554-3100-2800-0000-000000141c9f,76051,2822,TX,Grapevine,Woodland Hills Dr,USA,None,Dr,None,None,-97.108195,32.899526,tarrant,POINT (-97.10819 32.89953)
4,00005554-3100-2800-0000-000000145490,76051,937,TX,Grapevine,Honeysuckle,USA,None,None,None,None,-97.079841,32.950518,tarrant,POINT (-97.07984 32.95052)


In [13]:
# source_df = pd.read_pickle(f'wrang_source_v0_{county}.pickle')
# source_gdf = gpd.GeoDataFrame(source_df, geometry = 'geometry', crs='EPSG:4326')

### Ingesting Deltas

Here goes the table of new changes to make to coordinates:

In [14]:
print("DELTA for updating geometries is ",delta)
if delta:
    delta_table_read = pd.read_pickle(Updated_geometries_pickle)
    delta_table_read = delta_table_read[['feat_id', 'updated_geometries']]
    delta_table_read['datetime_version'] = pd.Timestamp.now(tz = 'utc')


DELTA for updating geometries is  True


In [15]:
delta_table_read.head()

,feat_id,updated_geometries,datetime_version
0,00005554-3100-2800-0000-0000000f7284,POINT (-97.54305 32.99252),2022-10-06 09:52:52.710087+00:00
1,00005554-3100-2800-0000-0000000f52c1,POINT (-97.54284 32.99371),2022-10-06 09:52:52.710087+00:00
2,00005554-3100-2800-0000-0000000f52c3,POINT (-97.54252 32.99370),2022-10-06 09:52:52.710087+00:00
3,00005554-3100-2800-0000-0000000f53f8,POINT (-97.52674 32.99366),2022-10-06 09:52:52.710087+00:00
4,e914d065-ee44-4b01-aa96-e9a15c10be8e,POINT (-97.52674 32.99366),2022-10-06 09:52:52.710087+00:00


In [16]:
if delta:
    delta_table_write = delta_table_read[['feat_id', 'updated_geometries', 'datetime_version']]
    delta_table_write['updated_geometries'] = delta_table_write['updated_geometries'].astype(str)

In [17]:
if delta:
    raw2p.write_to_db(df = delta_table_write, schema = 'STAN_169', table_name = 'delta_table')

Table stored!


### Reading Deltas

In [18]:
if delta:  
  delta_query = f"""
  SELECT * FROM "STAN_169".delta_table where county = '{county}'
  """
  delta_df = raw2p.read_from_db(query = delta_query)
  delta_geom = gpd.GeoSeries.from_wkt(delta_df.updated_geometries)
  delta_gdf = gpd.GeoDataFrame(delta_df.drop('updated_geometries', axis = 1),
                                  geometry = delta_geom,
                                  crs = 'EPSG:4326')

In [19]:
if delta:
  delta_gdf.head()

### Replacing Delta Changes

In [20]:
def replace_geometries(source_gdf, delta_gdf):
    '''
    Takes a 'source' geodataframe - copy of MNR database, a 'delta' geodataframe and replaces, for every APT (key: feat_id) in source dataframe, the 
    coordinates in the sorce with the coordinates in the new 

            Parameters:
                    source_gdf (gpd.GeoDataFrame): geodataframe containing MNR coordinates for every APT (feat_id)
                    delta_gdf (gpd.GeoDataFrame): geodataframe containing NEW coordinates for some APTs (feat_id)

            Returns:
                    source_gdf_new (gpd.GeoDataFrame): geodataframe containing MNR information for APT but with new coordinates
    '''
    # 
    delta_gdf_grouped = delta_gdf[delta_gdf.groupby('feat_id').datetime_version.transform('max') == delta_gdf.datetime_version]

    source_gdf_new = source_gdf.merge(delta_gdf_grouped[["feat_id", "geometry"]], on="feat_id", how="left")

    source_gdf_new.loc[~source_gdf_new.geometry_y.isna(), "geometry_x"] = source_gdf_new.loc[~source_gdf_new.geometry_y.isna(), "geometry_y"]

    source_gdf_new = source_gdf_new.drop(["geometry_y"], axis=1).rename({"geometry_x": "geometry"}, axis = 1)

    source_gdf_new = gpd.GeoDataFrame(source_gdf_new.drop('geometry', axis = 1), 
                                    geometry = source_gdf_new.geometry, crs = 'EPSG:4326')

    return source_gdf_new

In [21]:
if delta:
    source_delta = replace_geometries(source_gdf, delta_gdf)
    source_delta_gdf = gpd.GeoDataFrame(source_delta.drop('geometry', axis = 1), geometry = source_delta.geometry, 
                           crs = 'EPSG:4326')
    source_delta_gdf.x = source_delta_gdf.geometry.apply(lambda p: p.x)
    source_delta_gdf.y = source_delta_gdf.geometry.apply(lambda p: p.y)

### Joining Sample and Source - Spatial Join

In [22]:
if delta:
  joined_sample = source_delta_gdf.sjoin(sample_gdf, how='right', predicate='intersects')
else:
  joined_sample = source_gdf.sjoin(sample_gdf, how = 'right', predicate = 'intersects')

In [23]:
joined_sample.head()

,index_left,feat_id,postal_code,hsn,state,city,street_name,country_code,prefix,suffix,...,libpostal_state_district,libpostal_state,libpostal_country_region,libpostal_country,libpostal_world_region,lat_sample,lon_sample,county_right,sample_id,geometry
0,183,00005554-3100-2800-0000-00000012283d,76104,1219,TX,Fort Worth,Fairmount Ave,USA,None,Ave,...,,tx,,usa,,32.728714,-97.344899,tarrant,1ext3ggtarrant,"POLYGON ((-97.33592 32.72871, -97.33596 32.727..."
0,1331,00005554-3100-2800-0000-0000001288f7,76104,815,TX,Fort Worth,8th Ave,USA,None,Ave,...,,tx,,usa,,32.728714,-97.344899,tarrant,1ext3ggtarrant,"POLYGON ((-97.33592 32.72871, -97.33596 32.727..."
0,1545,00005554-3100-2800-0000-00000010e8f5,76110,1940,TX,Fort Worth,Berkeley Pl,USA,None,Pl,...,,tx,,usa,,32.728714,-97.344899,tarrant,1ext3ggtarrant,"POLYGON ((-97.33592 32.72871, -97.33596 32.727..."
0,2059,00005554-3100-2800-0000-000000122941,76104,1111,TX,Fort Worth,S Henderson St,USA,None,St,...,,tx,,usa,,32.728714,-97.344899,tarrant,1ext3ggtarrant,"POLYGON ((-97.33592 32.72871, -97.33596 32.727..."
0,2728,00005554-3100-2800-0000-0000001226ab,76110,2309,TX,Fort Worth,Mistletoe Ave,USA,None,Ave,...,,tx,,usa,,32.728714,-97.344899,tarrant,1ext3ggtarrant,"POLYGON ((-97.33592 32.72871, -97.33596 32.727..."


In [24]:
# if delta: 
#   del source_gdf
#   del delta_gdf
# else:
#   del source_delta_gdf

In [25]:
joined_sample.rename({
    'hsn': 'hsnum',
    'street_name': 'st_name',
    'postal_code': 'zip_code'
},
axis = 1, 
inplace = True)

# Parsing Joined Sample

In [26]:
def parse_joined_sample(spatial_joined_df: pd.DataFrame) -> pd.DataFrame:
    '''Function inversely parses the addresses to create a searched query format so that the addresses in the source 
    can be compared to the addresses in the sample.

    :param spatial_joined_df: DataFrame that contains the addresses from the source that are within the polygon of 
    the sample generated. It must contain the columns: ['hsn', 'unit_type', 'unit_num', 'pre_dir', 'prefix', 'suffix'
    'post_dir', 'city', 'state', 'zip_code']
    :type spatial_joined_df: pd.DataFrame
    :return: The same dataframe with a column that contains the full addresses inversely parsed.
    :rtype: pd.DataFrame
    '''

    df = spatial_joined_df.copy()

    dict_of_columns = {
        'hsnum': ' ', 'pre_dir': ' ', 'st_name': ' ', 'suffix': ', ', 'city': ' ', 'state': ' ', 'zip_code': ', ', 'country': ''
    }
    df['pre_dir'].fillna('', inplace=True)
    df['prefix'].fillna('', inplace=True)
    df['suffix'].fillna('', inplace=True)
    df['post_dir'].fillna('', inplace=True)

    for column in dict_of_columns.keys():

        df[column + '_modified'] = df[column].astype(str) + dict_of_columns[column]

    list_of_modified_columns = [col for col in df.columns if '_modified' in col]

    df['searched_query'] = df[list_of_modified_columns].sum(axis=1)

    df['street_name'] = df['pre_dir'] + ' ' + df['prefix'] + ' ' + df['st_name'] + ' ' + df['suffix'] + ' ' + df['post_dir']
    df['name'] = '' #df['state']

    df = df.rename(columns={
        'hsnum': 'hsn', 'searched_query': 'address', 'zip_code': 'postal_code', 'city': 'place_name', 
        'y': 'lat', 'x': 'lon'
    })

    return df

In [27]:
parsed_df = parse_joined_sample(joined_sample)
parsed_df.head()

,index_left,feat_id,postal_code,hsn,state,place_name,st_name,country_code,prefix,suffix,...,pre_dir_modified,st_name_modified,suffix_modified,city_modified,state_modified,zip_code_modified,country_modified,address,street_name,name
0,183,00005554-3100-2800-0000-00000012283d,76104,1219,TX,Fort Worth,Fairmount Ave,USA,,Ave,...,,Fairmount Ave,"Ave,",Fort Worth,TX,"76104,",USA,"1219 Fairmount Ave Ave, Fort Worth TX 76104, USA",Fairmount Ave Ave,
0,1331,00005554-3100-2800-0000-0000001288f7,76104,815,TX,Fort Worth,8th Ave,USA,,Ave,...,,8th Ave,"Ave,",Fort Worth,TX,"76104,",USA,"815 8th Ave Ave, Fort Worth TX 76104, USA",8th Ave Ave,
0,1545,00005554-3100-2800-0000-00000010e8f5,76110,1940,TX,Fort Worth,Berkeley Pl,USA,,Pl,...,,Berkeley Pl,"Pl,",Fort Worth,TX,"76110,",USA,"1940 Berkeley Pl Pl, Fort Worth TX 76110, USA",Berkeley Pl Pl,
0,2059,00005554-3100-2800-0000-000000122941,76104,1111,TX,Fort Worth,S Henderson St,USA,,St,...,S,S Henderson St,"St,",Fort Worth,TX,"76104,",USA,"1111 S S Henderson St St, Fort Worth TX 76104,...",S S Henderson St St,
0,2728,00005554-3100-2800-0000-0000001226ab,76110,2309,TX,Fort Worth,Mistletoe Ave,USA,,Ave,...,,Mistletoe Ave,"Ave,",Fort Worth,TX,"76110,",USA,"2309 Mistletoe Ave Ave, Fort Worth TX 76110, USA",Mistletoe Ave Ave,


In [28]:
del joined_sample

# Matching Adresses

In [29]:
def apt_similarity_filter(
    #country:str,
    df:pd.DataFrame,
    sample_df:pd.DataFrame,
    stopwords_pattern: str = '') -> pd.DataFrame:
    """Performs matching after making call in a given radius

    :param country: country to call in MNR
    :type country: str
    :param df: DataFrame containing the sample addresses (must have coordinates)
    :type df: pd.DataFrame
    :param sample_df: DataFrame containing libpostal components for sample (df) addresses
    :type sample_df: pd.DataFrame
    :param radius: radius of the buffer
    :type radius: float
    :param inner_radius: radius in meters of a smaller buffer. When bigger than zero, we are essentially getting the point in a disk, defaults to 0
    :type inner_radius: int or float, optional
    :param stopwords_pattern: regex pattern to remove stopwords, if needed. Optional, defaults to None
    :type stopwords_pattern: str
    :return: DataFrame with the APTs that matched
    :rtype: pd.DataFrame
    """
    apts_df = df.copy()

    # Fill NAs
    apts_df[['address', 'street_name', 'hsn', 'postal_code',
                    'place_name', 'name']] = apts_df[['address', 'street_name', 'hsn',
                                                                            'postal_code', 'place_name', 'name']].fillna('')

    # Drop duplicates
    #apts_df = apts_df.drop_duplicates(['searched_query', 'address']).reset_index(drop=True)
    
    
    # Create extra columns for stopwords, optional unidecode 
    cols_stopwords = ['address', 'street_name', 'place_name']
    for col in cols_stopwords:
        col_create = col + '_no_stopwords'
        apts_df[col_create] =  apts_df[col].str.replace(stopwords_pattern, '', case=False, regex=True)
        
    for col in cols_stopwords:
        col_create = col + '_no_stopwords_unidecode'
        apts_df[col_create] =  apts_df[col+'_no_stopwords'].apply(lambda x: unidecode.unidecode(x))
        
    
    # Merge to APTs
    #apts_df = apts_df.merge(sample_df.drop(columns=['country', 'searched_query_unidecode_sample']),
    #                                      how='left', 
    #                                      on=['searched_query'])
    apts_df['libpostal_road_no_stopwords'] = apts_df.libpostal_road.str.replace(stopwords_pattern, '', case=False, regex=True)


    # House number similarity: filter obvious non matches
    apts_df['hsn_similarity'] = list(map(fuzz.token_set_ratio, apts_df.libpostal_house_number, apts_df.hsn))
    apts_df['re_pattern'] = '\\b' + apts_df.hsn.astype(str) + '\\b'
    #apts_df['hsn_in_query'] = apts_df.apply(lambda x: bool(re.search(x.re_pattern, x.searched_query_unidecode_sample)), axis=1)
    #apts_df['hsn_similarity'] = np.where((apts_df.hsn_in_query), 100, apts_df.hsn_similarity)

    dropped_df = apts_df.loc[apts_df.hsn_similarity <= 60].reset_index(drop=True)
    
    
    apts_df = apts_df.loc[apts_df.hsn_similarity > 60].reset_index(drop=True)

    # Postal code similarity
    apts_df['postcode_similarity'] = list(map(fuzz.WRatio, 
                                                     apts_df.libpostal_postcode, 
                                                     apts_df.postal_code.fillna('').astype(str)))
    apts_df['postcode_similarity'] = np.where(apts_df.libpostal_postcode=='', np.nan,
                                                     np.where(apts_df.postal_code=='', 50, apts_df.postcode_similarity))

    
    # Road similarity
    apts_df['road_similarity'] = list(map(fuzz.token_set_ratio, 
                                                 apts_df.libpostal_road_no_stopwords, 
                                                 apts_df.street_name_no_stopwords))
    apts_df['road_similarity_unidecode'] = list(map(fuzz.token_set_ratio, 
                                                           apts_df.libpostal_road_no_stopwords, 
                                                           apts_df.street_name_no_stopwords_unidecode)) 
    apts_df['road_similarity'] = apts_df[['road_similarity', 'road_similarity_unidecode']].max(axis=1)
    
    # Locality similarity
    apts_df['searched_query_tokens'] = (apts_df.libpostal_road.astype(str) + ' ' + 
                                               apts_df.libpostal_house_number.astype(str) + ' ' + 
                                               apts_df.libpostal_postcode.astype(str))
    
    apts_df['provider_tokens'] = (apts_df.street_name.astype(str) + ' ' + 
                                         apts_df.hsn.astype(str) + ' ' + apts_df.postal_code.astype(str))
    apts_df['aux_searched_query'] = apts_df.apply(lambda x: automatic_matching.replace_tokens(x.searched_query_unidecode_sample, x.searched_query_tokens), axis=1)
    apts_df['aux_provider_address'] = apts_df.apply(lambda x: automatic_matching.replace_tokens(x.address, x.provider_tokens), axis=1)
    apts_df['aux_provider_address'] = apts_df.aux_provider_address.fillna('').apply(lambda x: unidecode.unidecode(x))
    apts_df['locality_wratio'] = apts_df.apply(lambda x: fuzz.WRatio(str(x.aux_searched_query).lower(), str(x.aux_provider_address).lower()), axis=1)
    apts_df['locality_city_state_ratio'] = apts_df.apply(lambda x: fuzz.WRatio(str(x.libpostal_city) + ' ' + str(x.libpostal_state),
                                                                                            str(x.place_name) + ' ' + str(x.name)), axis=1)
    apts_df['locality_similarity'] = apts_df[['locality_wratio', 'locality_city_state_ratio']].mean(axis=1)

    apts_df['mnr_query_distance'] = apts_df.apply(lambda x: haversine_distance(x.lat, x.lon,
                                                                                               x.lat_sample, x.lon_sample)
                                                                  if not np.isnan(x.lat) else 1e7
                                                                  , axis=1)

    # Compute mean similarity
    apts_df['mean_similarity'] = (apts_df[['locality_similarity', 'hsn_similarity', 
                                                         'postcode_similarity', 'road_similarity']].mean(axis=1)
                                        * np.where(apts_df.hsn_similarity >= 70 , 1, 0)
                                        * np.where(apts_df.road_similarity >= 60 , 1, 0)  
                                        * np.where(apts_df.mnr_query_distance > 1000, 0, 1)
                                        )


    apts_df_matching = (
        apts_df.sort_values(by='mnr_query_distance')
        .loc[apts_df.groupby(['sample_id'])
        .mean_similarity.idxmax()]
        .reset_index(drop=True)
    )


    # Compute matching
    apts_df_matching['match'] = pd.NaT
    
    apts_df_matching['match'] = np.where(apts_df_matching.mean_similarity >= 70, 1, pd.NaT) #90 so far best

    #address_matches = apts_df_matching['searched_query_unidecode_sample']
    address_sample_ids = apts_df_matching['sample_id']
    
    #non_matches = dropped_df[~dropped_df['searched_query_unidecode_sample'].isin(address_matches)]
    non_matches_ids = dropped_df[~dropped_df['sample_id'].isin(address_sample_ids)]
    
    #addresses_to_add = non_matches['searched_query_unidecode_sample'].unique()
    addresses_to_add_ids = non_matches_ids['sample_id'].unique()
     
    addresses_id_df = pd.DataFrame(
        {'sample_id': addresses_to_add_ids, 'match': [pd.NaT] * len(addresses_to_add_ids)}
    )
    
    addresses_id_df = addresses_id_df.merge(sample_df[['sample_id', 'searched_query_unidecode_sample']], on = 'sample_id', how = 'left')
    
    cols_to_add = [col for col in apts_df_matching if col not in addresses_id_df.columns]
    
    addresses_id_df.loc[:, cols_to_add] = ''
    addresses_id_df_reordered = addresses_id_df[apts_df_matching.columns]
    
    apts_final = pd.concat([apts_df_matching, addresses_id_df_reordered])
    

    return apts_final

In [30]:
similarity_df = apt_similarity_filter(df = parsed_df, sample_df = sample_gdf, stopwords_pattern = countries_stopwords.get('us'))
del sample_gdf
del parsed_df

In [31]:
similarity_df['match'] = similarity_df['match'].fillna(0)

match_proportion = np.mean(similarity_df['match'])
clean_proportion = round(match_proportion * 100, 2)
print(f'The proportion of matches is: {clean_proportion}%')

The proportion of matches is: 94.39%


In [32]:
similarity_df.head()

,index_left,feat_id,postal_code,hsn,state,place_name,st_name,country_code,prefix,suffix,...,searched_query_tokens,provider_tokens,aux_searched_query,aux_provider_address,locality_wratio,locality_city_state_ratio,locality_similarity,mnr_query_distance,mean_similarity,match
0,60083,00005554-3100-2800-0000-000000136025,76118,7232,TX,Fort Worth,Brooks Ave,USA,,Ave,...,brooks ave 7232 76118,Brooks Ave Ave 7232 76118,richland hills tx usa,fort worth tx usa,50,18,34.0,2.0,83.5,1
1,115388,00005554-3100-2800-0000-00000010d22b,76126,1121,TX,Fort Worth,Manning St,USA,,St,...,manning st 1121 76126,Manning St St 1121 76126,benbrook tx usa,fort worth tx usa,59,22,40.5,3.1,85.125,1
2,392610,00005554-3100-2800-0000-00000010898c,76008,11913,TX,Aledo,Camp Bowie West Blvd,USA,,Blvd,...,camp bowie w blvd 11991 76008,Camp Bowie West Blvd Blvd 11913 76008,aledo tx usa,aledo tx usa,100,73,86.5,44.0,90.125,1
3,55614,00005554-3100-2800-0000-000000101e9f,76106,2540,TX,Fort Worth,Meacham Blvd,USA,,Blvd,...,meacham blvd 2540 76106,Meacham Blvd Blvd 2540 76106,fort worth tx usa,fort worth tx usa,100,83,91.5,21.6,97.875,1
4,180096,00005554-3100-2800-0000-000000228ab8,76244,3960,TX,Keller,Grizzly Hills Cir,USA,,Cir,...,grizzly hills cir 3960 76244,Grizzly Hills Cir Cir 3960 76244,fort worth tx usa,keller tx usa,60,16,38.0,2.5,84.5,1


In [33]:
match_df = similarity_df[['feat_id', 'match', 'sample_id']]
match_df['county'] = county
match_df['datetime_run'] = pd.Timestamp.now(tz = 'utc')
match_df.rename({'match': 'asf'}, axis = 1, inplace = True)

/tmp/ipykernel_559/2013992509.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match_df['county'] = county
/tmp/ipykernel_559/2013992509.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match_df['datetime_run'] = pd.Timestamp.now(tz = 'utc')
/home/tandon/anaconda3/envs/pygeo/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

In [34]:
match_df.head()

,feat_id,asf,sample_id,county,datetime_run
0,00005554-3100-2800-0000-000000136025,1,1000ext3ggtarrant,tarrant,2022-10-06 10:10:43.432018+00:00
1,00005554-3100-2800-0000-00000010d22b,1,1001ext3ggtarrant,tarrant,2022-10-06 10:10:43.432018+00:00
2,00005554-3100-2800-0000-00000010898c,1,1002ext3ggtarrant,tarrant,2022-10-06 10:10:43.432018+00:00
3,00005554-3100-2800-0000-000000101e9f,1,1003ext3ggtarrant,tarrant,2022-10-06 10:10:43.432018+00:00
4,00005554-3100-2800-0000-000000228ab8,1,1004ext3ggtarrant,tarrant,2022-10-06 10:10:43.432018+00:00


#### Bootstrapping

In [35]:
def bootstrap_resample(df, agg_fun, times=1000, seed=0):
    reboot = []
    
    for t in range(times):
        df_boot = df.sample(frac = 1, replace=True, random_state = t+seed)
        reboot.append(agg_fun(df_boot))
        
    return reboot


def percentile_bootstrap(df, agg_fun, conf=0.9, times=1000, seed=0):
    """Generic Percentile Bootstrap
    This function returns a percentile bootstrap confidence interval for a statistic.
    Args:
        df (pandas.DataFrame): DataFrame with the observed random vectors. Each row represents an observation an each column is a random variable.
        agg_fun (function): Aggregation function. This function should receive as input a pandas.DataFrame (resamples) and return a 
        number with the computed statistic.
        conf (float, optional): Confidence level of the returned interval. Defaults to 0.9.
        times (int, optional): Bootstrap resamples. Defaults to 1000.
        seed (int, optional): Random seed. Defaults to 0.
    Returns:
        numpy.array: Percentile Boostrap CI [lower, upper]
    """    
    reboot = bootstrap_resample(df, agg_fun, times, seed)
    return np.quantile(reboot, [(1-conf)/2, (1-conf)/2+conf])

In [36]:
[lower_distance, upper_distance] = percentile_bootstrap(similarity_df['match'], np.mean)

In [37]:
if delta:
    version = date
else:
    version = 'New_benchmark'

results_sum = pd.DataFrame(
    data=[[lower_distance, match_proportion, upper_distance, '%', 'ASF', version, county]], 
    columns=['lower_bound', 'calculated_metric', 'upper_bound', 'units', 'metric', 'version', 'county'], index = None)
results_sum

,lower_bound,calculated_metric,upper_bound,units,metric,version,county
0,0.93865,0.943939,0.949228,%,ASF,2022-10-06,tarrant


## Positional Accuracy

### 90th percentile

We consider the 90th percentile of the distance of matches as a metric for the Positional Accuracy. The distance we obtain below is be the distance for which 90% of the data is lower. The interesting thing about this metric is that it's expressed in terms of distance.

In [38]:
matches_df = similarity_df[similarity_df['match'] == 1]

matches_df['mnr_query_distance'] = matches_df['mnr_query_distance'].astype(float)

del similarity_df

/tmp/ipykernel_559/1420054033.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches_df['mnr_query_distance'] = matches_df['mnr_query_distance'].astype(float)


In [39]:
positional_accuracy_distance = round(np.quantile(matches_df['mnr_query_distance'], 0.9), 2)
print(f'Positional Accuracy (90th percentile distance) is: {positional_accuracy_distance}m')

Positional Accuracy (90th percentile distance) is: 61.39m


#### Bootstrapping

In [40]:
[lower_percentile90, upper_percentile90] = percentile_bootstrap(
    matches_df['mnr_query_distance'], lambda x: np.quantile(x, 0.9)
)

In [41]:
if delta:
    version = date
else:
    version = 'New_benchmark'

new_result = pd.DataFrame(
    data=[[lower_percentile90, positional_accuracy_distance, upper_percentile90, 'meters', '90p', version, county]], 
    columns=['lower_bound', 'calculated_metric', 'upper_bound', 'units', 'metric', 'version', 'county'], index = None)
results_sum = pd.concat([results_sum, new_result])
results_sum

,lower_bound,calculated_metric,upper_bound,units,metric,version,county
0,0.93865,0.943939,0.949228,%,ASF,2022-10-06,tarrant
0,57.87200,61.390000,66.200000,meters,90p,2022-10-06,tarrant


### % of matches below 50m

In [42]:
proportion_50m_matches = (matches_df['mnr_query_distance'] <= 50).mean()
nice_num_50m = round(proportion_50m_matches * 100, 1)
print(f'The calculated percentage of matches within 50 meters is {nice_num_50m}%')

The calculated percentage of matches within 50 meters is 88.1%


In [43]:
[lower_50m_pa, upper_50m_pa] = percentile_bootstrap(
    matches_df['mnr_query_distance'] <= 50, np.mean
)

In [44]:
if delta:
    version = date
else:
    version = 'New_benchmark'

new_result = pd.DataFrame(
     data=[[lower_50m_pa, proportion_50m_matches, upper_50m_pa, '%', 'APA', version, county]], 
    columns=['lower_bound', 'calculated_metric', 'upper_bound', 'units', 'metric', 'version', 'county'])
results_sum = pd.concat([results_sum, new_result])
results_sum

,lower_bound,calculated_metric,upper_bound,units,metric,version,county
0,0.938650,0.943939,0.949228,%,ASF,2022-10-06,tarrant
0,57.872000,61.390000,66.200000,meters,90p,2022-10-06,tarrant
0,0.873364,0.881219,0.888839,%,APA,2022-10-06,tarrant


In [45]:
# set(sample_gdf.sample_id) == set(similarity_df.sample_id)

# Join Matches Table

In [46]:
matches_df[['sample_id', 'mnr_query_distance']]

,sample_id,mnr_query_distance
0,1000ext3ggtarrant,2.0
1,1001ext3ggtarrant,3.1
2,1002ext3ggtarrant,44.0
3,1003ext3ggtarrant,21.6
4,1004ext3ggtarrant,2.5
...,...,...
4718,997ext3ggtarrant,2.7
4719,998ext3ggtarrant,20.8
4720,999ext3ggtarrant,2.9
4721,99ext3ggtarrant,1.0


In [47]:
match_df = match_df.merge(matches_df[['sample_id', 'mnr_query_distance']], on = ['sample_id'], how = 'left')
match_df['apa'] = match_df.mnr_query_distance.apply(lambda x: 1 if x < 50 else 0)
match_df.drop('mnr_query_distance', axis = 1, inplace = True)
# raw2p.write_to_db(match_df, table_name = 'matches_table', schema = 'STAN_169')

In [48]:
if delta:
  match_df = match_df.merge(source_delta_gdf[['feat_id', 'geometry']], on = ['feat_id'], how = 'inner')
  del source_delta_gdf
else:
  match_df = match_df.merge(source_gdf[['feat_id', 'geometry']], on = ['feat_id'], how = 'inner')
  del source_gdf

In [49]:
match_df.head()

,feat_id,asf,sample_id,county,datetime_run,apa,geometry
0,00005554-3100-2800-0000-000000136025,1,1000ext3ggtarrant,tarrant,2022-10-06 10:10:43.432018+00:00,1,POINT (-97.22314 32.81418)
1,00005554-3100-2800-0000-00000010d22b,1,1001ext3ggtarrant,tarrant,2022-10-06 10:10:43.432018+00:00,1,POINT (-97.45116 32.67120)
2,00005554-3100-2800-0000-00000010898c,1,1002ext3ggtarrant,tarrant,2022-10-06 10:10:43.432018+00:00,1,POINT (-97.52983 32.71980)
3,00005554-3100-2800-0000-000000101e9f,1,1003ext3ggtarrant,tarrant,2022-10-06 10:10:43.432018+00:00,1,POINT (-97.31483 32.82375)
4,00005554-3100-2800-0000-000000228ab8,1,1004ext3ggtarrant,tarrant,2022-10-06 10:10:43.432018+00:00,1,POINT (-97.27464 32.93593)


In [50]:
match_df.geometry = match_df.geometry.astype(str)

# Store Results @ psql

### Storing Results

In [51]:
results_sum

,lower_bound,calculated_metric,upper_bound,units,metric,version,county
0,0.938650,0.943939,0.949228,%,ASF,2022-10-06,tarrant
0,57.872000,61.390000,66.200000,meters,90p,2022-10-06,tarrant
0,0.873364,0.881219,0.888839,%,APA,2022-10-06,tarrant


In [52]:
raw2p.write_to_db(results_sum, table_name = 'results', schema = 'STAN_169')

Table stored!


1

### Storing Matches

In [53]:
match_df['version'] = date

In [54]:
match_df.head()

,feat_id,asf,sample_id,county,datetime_run,apa,geometry,version
0,00005554-3100-2800-0000-000000136025,1,1000ext3ggtarrant,tarrant,2022-10-06 10:10:43.432018+00:00,1,POINT (-97.223138 32.814181),2022-10-06
1,00005554-3100-2800-0000-00000010d22b,1,1001ext3ggtarrant,tarrant,2022-10-06 10:10:43.432018+00:00,1,POINT (-97.451159 32.6712),2022-10-06
2,00005554-3100-2800-0000-00000010898c,1,1002ext3ggtarrant,tarrant,2022-10-06 10:10:43.432018+00:00,1,POINT (-97.529825 32.719796),2022-10-06
3,00005554-3100-2800-0000-000000101e9f,1,1003ext3ggtarrant,tarrant,2022-10-06 10:10:43.432018+00:00,1,POINT (-97.314832 32.823755),2022-10-06
4,00005554-3100-2800-0000-000000228ab8,1,1004ext3ggtarrant,tarrant,2022-10-06 10:10:43.432018+00:00,1,POINT (-97.274644 32.935926),2022-10-06


In [55]:
from shapely import wkt 
# raw2p.write_to_db(match_df, table_name = 'matches_table', schema = 'STAN_169')
# match_df.to_csv(os.path.join(data_path,state,county,'Apt_realignment_MSFT/matching_df.csv'))
match_df['geometry'] = match_df['geometry'].apply(wkt.loads)

In [61]:
gdf = gpd.GeoDataFrame(match_df[['feat_id','sample_id','apa','asf','geometry']], crs="epsg:4326", geometry='geometry')
gdf.to_crs("epsg:4326")
gdf.to_file(os.path.join(data_path,state,county,'Apt_realignment_MSFT/matching_df.shp'), driver='ESRI Shapefile')

In [62]:
match_df

,feat_id,asf,sample_id,county,datetime_run,apa,geometry,version
0,00005554-3100-2800-0000-000000136025,1,1000ext3ggtarrant,tarrant,2022-10-06 10:10:43.432018+00:00,1,POINT (-97.22314 32.81418),2022-10-06
1,00005554-3100-2800-0000-00000010d22b,1,1001ext3ggtarrant,tarrant,2022-10-06 10:10:43.432018+00:00,1,POINT (-97.45116 32.67120),2022-10-06
2,00005554-3100-2800-0000-00000010898c,1,1002ext3ggtarrant,tarrant,2022-10-06 10:10:43.432018+00:00,1,POINT (-97.52983 32.71980),2022-10-06
3,00005554-3100-2800-0000-000000101e9f,1,1003ext3ggtarrant,tarrant,2022-10-06 10:10:43.432018+00:00,1,POINT (-97.31483 32.82375),2022-10-06
4,00005554-3100-2800-0000-000000228ab8,1,1004ext3ggtarrant,tarrant,2022-10-06 10:10:43.432018+00:00,1,POINT (-97.27464 32.93593),2022-10-06
...,...,...,...,...,...,...,...,...
4718,00005554-3100-2800-0000-00000011f46a,1,997ext3ggtarrant,tarrant,2022-10-06 10:10:43.432018+00:00,1,POINT (-97.24797 32.66490),2022-10-06
4719,bb2a6070-4259-41bc-8505-06261138e312,1,998ext3ggtarrant,tarrant,2022-10-06 10:10:43.432018+00:00,1,POINT (-97.41069 32.65425),2022-10-06
4720,00005554-3100-2800-0000-00000013a48d,1,999ext3ggtarrant,tarrant,2022-10-06 10:10:43.432018+00:00,1,POINT (-97.23602 32.86534),2022-10-06
4721,00005554-3100-2800-0000-0000001c9a58,1,99ext3ggtarrant,tarrant,2022-10-06 10:10:43.432018+00:00,1,POINT (-97.50961 32.74545),2022-10-06
